# Exploration of Loglikelihood computations

This notebook is not running yet

In [ ]:
X[0]

tensor(1)

In [ ]:
k.loglikelihood(X, smooth=False)

tensor(-3.7474)

In [ ]:
from scipy import linalg

In [ ]:
def log_multivariate_normal_density(X, means, covars, min_covar=1.e-7):
    """Log probability for full covariance matrices. """
    if hasattr(linalg, 'solve_triangular'):
        # only in scipy since 0.9
        solve_triangular = linalg.solve_triangular
    else:
        # slower, but works
        solve_triangular = linalg.solve
    n_samples, n_dim = X.shape
    nmix = len(means)
    log_prob = np.empty((n_samples, nmix))
    for c, (mu, cv) in enumerate(zip(means, covars)):
        try:
            cv_chol = linalg.cholesky(cv, lower=True)
        except linalg.LinAlgError:
            # The model is most probabily stuck in a component with too
            # few observations, we need to reinitialize this components
            cv_chol = linalg.cholesky(cv + min_covar * np.eye(n_dim),
                                      lower=True)
        cv_log_det = 2 * np.sum(np.log(np.diagonal(cv_chol)))
        cv_sol = solve_triangular(cv_chol, (X - mu).T, lower=True).T
        log_prob[:, c] = - .5 * (np.sum(cv_sol ** 2, axis=1) + \
                                     n_dim * np.log(2 * np.pi) + cv_log_det)

    return log_prob

In [ ]:
log_multivariate_normal_density(np.ones((1,1)), np.zeros(1), np.eye(1))

array([[-1.41893853]])

In [ ]:
MultivariateNormal(torch.zeros(1), torch.eye(1)).log_prob(torch.ones((1)))

tensor(-1.4189)

In [ ]:
log_multivariate_normal_density(np.ones((1, 2)), np.zeros((1,2)), np.expand_dims(np.eye(2), 0))

array([[-2.83787707]])

In [ ]:
MultivariateNormal(torch.zeros(2), torch.eye(2)).log_prob(torch.ones((2)))

tensor(-2.8379)

pytorch and pykalman have the same results for computing the loglikelihood of a gaussian distribution

In [ ]:
def _torch_loglikelihoods(observation_matrices, observation_offsets,
                    observation_covariance, predicted_state_means,
                    predicted_state_covariances, observations):
    
    n_timesteps = observations.shape[0]
    loglikelihoods = np.zeros(n_timesteps)
    for t in range(n_timesteps):
        observation = observations[t]
        observation_matrix = _last_dims(observation_matrices, t)
        observation_offset = _last_dims(observation_offsets, t, ndims=1)
        predicted_state_mean = _last_dims(
            predicted_state_means, t, ndims=1
        )
        predicted_state_covariance = _last_dims(
            predicted_state_covariances, t
        )

        predicted_observation_mean = (
            observation_matrix @ predicted_state_mean
            + observation_offset
        )
        predicted_observation_covariance = (
            observation_matrix @ predicted_state_covariance @ observation_matrix.T
            + observation_covariance
        )

        loglikelihoods[t] = MultivariateNormal(
            predicted_observation_mean,
            predicted_observation_covariance
        ).log_prob(observation.unsqueeze(0))
    return loglikelihoods.sum()

In [ ]:
state = k.filter(X)

In [ ]:
pyk.loglikelihood??

Signature: pyk.loglikelihood(X)
Source:   
    def loglikelihood(self, X):
        """Calculate the log likelihood of all observations

        Parameters
        ----------
        X : [n_timesteps, n_dim_obs] array
            observations for time steps [0...n_timesteps-1]

        Returns
        -------
        likelihood : float
            likelihood of all observations
        """
        Z = np.array(self._parse_observations(X))

        # initialize parameters
        (transition_matrices, transition_offsets,
         transition_covariance, observation_matrices,
         observation_offsets, observation_covariance,
         initial_state_mean, initial_state_covariance) = (
            self._initialize_parameters()
        )

        # apply the Kalman Filter
        (predicted_state_means, predicted_state_covariances,
         kalman_gains, filtered_state_means,
         filtered_state_covariances) = (
            _filter(
                transition_matrices, observation_matr

In [ ]:
nX = X.numpy()

In [ ]:
pred_mean, pred_cov = pyk.filter(nX)

In [ ]:
pyk.loglikelihood(nX)

-5.231597970652478

In [ ]:
Z = np.array(pyk._parse_observations(nX))

In [ ]:
(transition_matrices, transition_offsets,
         transition_covariance, observation_matrices,
         observation_offsets, observation_covariance,
         initial_state_mean, initial_state_covariance) = (
            pyk._initialize_parameters()
        )

In [ ]:
(predicted_state_means, predicted_state_covariances,
         kalman_gains, filtered_state_means,
         filtered_state_covariances) = (
            pykalman.standard._filter(
                transition_matrices, observation_matrices,
                transition_covariance, observation_covariance,
                transition_offsets, observation_offsets,
                initial_state_mean, initial_state_covariance,
                Z
            )
        )

In [ ]:
pykalman.standard._loglikelihoods(
 observation_matrices, observation_offsets, observation_covariance,
          pred_mean, pred_cov, Z
).sum()

-3.7473859589713614

In [ ]:
_torch_loglikelihoods(
    k.obs_matrices,
    k.obs_offsets,
    k.obs_cov,
    state.mean,
    state.cov,
    X
)

-3.747385859489441

In [ ]:
pykalman.standard._loglikelihoods(
    k.obs_matrices.numpy(),
    k.obs_offsets.numpy(),
    k.obs_cov.numpy(),
    state.mean.numpy(),
    state.cov.numpy(),
    Z
).sum()

-3.747385949780805

In [ ]:
pyk.loglikelihood(Z)

-5.231597970652478

In [ ]:
test_close(pyk.filter(nX), k.filter(X))

In [ ]:
pyk.loglikelihood(nX)

-5.231597970652478

## Multivariate likelihood

In [ ]:
from torch.distributions import MultivariateNormal
import torch
from meteo_imp.gaussian import to_posdef

In [ ]:
n = 5

In [ ]:
cov = to_posdef(torch.rand(n,n))
mean = torch.rand(n)

In [ ]:
dist = MultivariateNormal(mean, cov)

In [ ]:
obs = dist.sample()

In [ ]:
obs

tensor([1.0237, 0.9482, 0.9448, 1.5264, 1.2376])

In [ ]:
dist.log_prob(obs)

tensor(-3.6350)

In [ ]:
dist2 = MultivariateNormal(mean, torch.diag(cov.diag()))

In [ ]:
dist2.covariance_matrix

tensor([[0.9555, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.1688, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.7292, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 2.1713, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0093]])

In [ ]:
dist2.log_prob(obs)

tensor(-6.6010)

so we see that if there is a diagonal covariance the likelihood is the same of the sum of the likelihoods of the individual variables

In [ ]:
tot = 0
for i in range(n):
    dist_n = MultivariateNormal(mean[i:i+1], cov[i:i+1, i:i+1])
    tot += dist_n.log_prob(obs[i:i+1])

In [ ]:
tot

tensor(-6.6010)

In [ ]:
dist_n.log_prob(obs[i:i+1])

tensor(-1.3723)

performance big ll vs individual

In [ ]:
n = 5000

In [ ]:
std = torch.rand(n)
mean = torch.rand(n)

In [ ]:
dist = MultivariateNormal(mean, torch.diag(std))

In [ ]:
obs = dist.sample()

In [ ]:
%timeit dist.log_prob(obs)

2.61 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
dist.log_prob(obs)

tensor(-4552.8486)

In [ ]:
def single_ll():
    tot = 0
    for i in range(n):
        dist_n = MultivariateNormal(mean[i:i+1], torch.diag(std[i:i+1]))
        tot += dist_n.log_prob(obs[i:i+1])

In [ ]:
tot

tensor(-6.6010)

In [ ]:
%timeit single_ll()

1.49 s ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
dist = MultivariateNormal(mean.cuda(), torch.diag(std).cuda())

In [ ]:
obs = dist.sample()

In [ ]:
%timeit dist.log_prob(obs)

630 µs ± 3.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
